In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy
import cv2
import pandas as pd


from render import Renderer
# Renderer is a class(package) which helps to plot in cv2
class object(Renderer):
    def __init__(self,data, recordLocation = None):
        super().__init__(recordLocation=recordLocation)
        self.its = 0
        self.k = 22   # stiffness
        self.q1 = 0     # q1 is the angle made by the first limb of manipulator with x axis
        self.q2 = 0     # q2 is the angle made by the second limb of manipulator with x axis
        self.theta = 0  # theta is the angle made by the limb 2 w.r.t to limb 1
        self.l1 = 200   # length of the first limb
        self.l2 = 380   # length of the second limb
        self.m1 = 5     # mass of the first limb
        self.m2 = 7     # mass of the seccond limb
        self.g =10    # gravity
        self.kut = 1  # kut denotes the index of other end position of spring
        self.points = set()
        self.data = data
    def getInfo(self):
        info = { 
            't1s': round(self.t1s,4),
            't2s': round(self.t2s,4),
            't1': round(self.t1,4),
            't2': round(self.t2,4),
            'total_t1':round(self.total_t1,4),
            'total_t2':round(self.total_t2,4),
            'x': round(self.x,4),
            'y': round(self.y,4),
            'kut': round(self.kut,1)
        }
        return info
    def plot(self):
        pass
    def inner(self,x,y,x_dot,y_dot):
        
            self.theta = math.acos(((x)**2+(y)**2-(self.l1)**2-(self.l2)**2)/(2*self.l1*self.l2)) 
            
            self.q1=math.atan((y)/(x))-math.atan((self.l2*math.sin(self.theta))/(self.l1+(self.l2*math.cos(self.theta))))
            self.q2=(self.q1+self.theta)
        
            self.Fx = (self.k*(self.l1*math.cos(self.q1)+self.l2*math.cos(self.q2) - self.data[0,0]))/1000 
            self.Fy = (self.k*(self.l1*math.sin(self.q1)+self.l2*math.sin(self.q2) - self.data[0,1]))/1000
        
            self.t1s = (self.Fy*self.l1*math.cos(self.q1)-self.Fx*self.l1*math.sin(self.q1))/1000 # torque provided at joint 1 due to Fx
            self.t2s = (self.Fy*self.l2*math.cos(self.q2)-self.Fx*self.l2*math.sin(self.q2))/1000 # torque provided at joint 2 due to Fy
            
            # q1_dot and q2_dot are the rate of change of angle at joint 1 and joint 2 respectively
            self.q1_dot = (self.l2*math.cos(self.q2)*x_dot + self.l2*math.sin(self.q2)*y_dot)/(self.l1*self.l2*math.sin(self.q2-self.q1))
            self.q2_dot = (-1*self.l1*math.cos(self.q1)*x_dot - self.l1*math.sin(self.q1)*y_dot)/(self.l1*self.l2*math.sin(self.q2-self.q1))
            
            # q1_ddot and q2_ddot are the rate of change of angular velocit at joint 1 and at joint 2 respectively
            self.q2_ddot = ((self.q1_dot)**2*self.l1 + (self.q2_dot)**2*math.cos(self.q2-self.q1)*self.l2)/self.l2*math.sin(self.q1-self.q2)
            self.q1_ddot = -1*(self.l1*math.cos(self.q1)*(self.q1_dot)**2 + self.l2*math.cos(self.q2)*(self.q2_dot)**2 +self.l2*math.sin(self.q2)*self.q2_ddot)/(self.l1*math.sin(self.q1))
            
            # torque provided at joint 1 due to dynamics of the limb 1
            self.t1 = ((1/3*self.m1+self.m2)*((self.l1)**2)*self.q1_ddot+1/2*self.m2*self.l1*self.l2*self.q2_ddot*math.cos(self.q2-self.q1)-1/2*self.m2*self.l1*self.l2*self.q2_dot*(self.q2_dot-self.q1_dot)*math.sin(self.q2-self.q1))/1000000 + ((1/2*self.m1+self.m2)*self.g*self.l1*math.cos(self.q1))/1000
            
            # torque provided at joint 2 due to dynamics of the limb 2
            self.t2 = ((1/3*self.m2+1/4*self.m2)*((self.l2)**2)*self.q2_ddot + 1/2*self.m2*self.l1*self.l2*self.q1_ddot*math.cos(self.q2-self.q1)-1/2*self.m2*self.l1*self.l2*self.q1_dot*(self.q2_dot-self.q1_dot)*math.sin(self.q2-self.q1))/1000000 + (1/2*self.m2*self.g*self.l2*math.sin(self.q2))/1000
        
            # total torque provided at joint 1 
            self.total_t1 = self.t1s + self.t1
            # total torque provided at joint 1
            self.total_t2 = self.t2s + self.t2
            
            # (x1,y1) is the coordinate of the joint 1
            self.x1 = self.l1*math.cos(self.q1)
            self.y1 = self.l1*math.sin(self.q1)
            
            # (x2,y2) is the coordinate of the joint 2
            self.x2 = self.x1+self.l2*math.cos(self.q2)
            self.y2 = self.y1+self.l2*math.sin(self.q2)
            
            anim.render(height = 600,pause = 10)
    def factor(self,i):
            if i%2==0:
                self.its = 0
            else:
                if i == self.kut:
                    self.its =1
                    self.kut +=4
                else:
                    self.its = 2
            return self.its
        
    def step(self,i,p):
        self.x_dot = 1500/p
        self.y_dot = 1500/p
        self.row = self.factor(i)    
        self.x = self.data[self.row,0]
        self.y = self.data[self.row,1]
        self.xint = self.x
        self.yint = self.y
        
        self.ro = self.factor(i+1)
        self.xfin = self.data[self.ro,0]
        self.yfin = self.data[self.ro,1]
        if i+5 ==self.kut:
            self.kut= self.kut-4
        self.k1 = (self.xfin-self.xint)/p
        self.k2 = (self.yfin-self.yint)/p
        
        
        #self.inner(self.x,self.y)
        for i in range(0,p):
            self.x += self.k1
            self.y += self.k2
            self.inner(self.x,self.y,self.x_dot,self.y_dot)
        
    def draw(self,image):

        cv2.line(image,(0,0),(int(self.x1),int(self.y1)),(0,255,0),2)
        cv2.line(image,(int(self.x1),int(self.y1)),(int(self.x2),int(self.y2)),(0,0,255),2)
        cv2.circle(image,(int(self.x2),int(self.y2)),2,(255,0,0),-1)  # trajectory traced by our manipulator
        return image      
user = list(map(int,input(" Enter the mean position(x0,y0) and desired position(x,y)(like- 279 400 279 500)").split()))
user.append(2*user[0]-user[2])
user.append(2*user[1]-user[3])
data = np.array(user)
data = data.reshape(3,2)
anim = object(data, recordLocation ='anim.mp4')
speed = int(input(" Enter the speed of end-effector"))
if 1500/speed>=15: 
    p = 100          # here 100 is the no. of points must be taken b/w two points
else:
    p = 1500/speed
    
for i in range(0,31):
    anim.step(i,p)
    
    

    
        
        

 Enter the mean position(x0,y0) and desired position(x,y)(like- 279 400 279 500)279 400 279 500
 Enter the speed of end-effector100
